# SustAInable MLflow Workshop

Willkommen! In diesem Notebook bauen wir einen kompakten **Train–Test–Deploy**-Workflow mit **MLflow** – inkl. **CO₂-Tracking** mit CodeCarbon und inline-UI.

**Ablauf:**
1. **Setup & UI** – Lokales Tracking (SQLite), MLflow-UI im Notebook  
2. **Daten** – Gesamtdatensatz & Train/Val/Test-Splits loggen  
3. **Training** – RandomForest + Metriken, Artefakte, Modell  
4. **CO₂-Tracking** – Emissionen pro Run messen  
5. **HPO** – Parent-Run + Child-Runs vergleichen  
6. **Anreicherung** – Feature Importances nachträglich loggen  
7. **Registry** – Bestes Modell registrieren & auf *Staging* promoten  
8. **Serving** – REST-API starten & Predictions testen  


## 0) Setup
**Nötige Environment aufsetzen**
# Optional – ausführen, falls noch nicht getan

Per one-liner:
```bash
conda create -n sustainable-mlflow python=3.10 \
  numpy pandas matplotlib scikit-learn mlflow \
  pytorch torchvision torchaudio cpuonly \
  plotly requests ipywidgets notebook codecarbon \
  -c pytorch -c conda-forge
```
oder per env.yml:
```bash
conda env create -f env.yml
```


**Die Biliotheken Importieren**

In [ ]:
# Die nötigen Bibliotheken importieren
import os
from pathlib import Path
import requests, json
import subprocess, time
from IPython.display import IFrame, display

import numpy as np
import pandas as pd

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

from codecarbon import EmissionsTracker

from matplotlib import pyplot as plt

/home/kai/.conda/envs/sustainable-mlflow/lib/python3.10/site-packages/codecarbon/input.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/kai/.conda/envs/sustainable-mlflow/lib/python3.10/site-packages/codecarbon/core/gpu.py:22: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml


In [2]:
# Suppress all FutureWarnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)



## 1) MLflow konfigurieren (lokal, SQLite mit Model Registry)
Wir nutzen eine **SQLite‑Datenbank** als Tracking‑Backend – Das ist aber nur eine von vielen Möglichkeiten.


In [3]:
# Lokale SQLite-DB (Datei wird im aktuellen Arbeitsverzeichnis angelegt)
MLFLOW_DB = "sqlite:///mlflow_workshop.db"
mlflow.set_tracking_uri(MLFLOW_DB)

## 1.1) MFLow GUI öffnen

In der GUI wir sehen was passiert ist, nicht zwingend notwendig. Aber sehr hilfreich.

In [ ]:
# disable new registry UI
os.environ["MLFLOW_ENABLE_NEW_REGISTRY_UI"] = "false"

# Startet die MLflow UI im Hintergrund auf Port 5003 (standard ist 5000)
# mit der SQLite DB als Backend-Store, falls man seine Daten wo anders speichert muss der Pfad angepasst werden
ui_proc = subprocess.Popen(
    ["mlflow", "ui", "--backend-store-uri", "sqlite:///mlflow_workshop.db", "-p", "5003"]
)

2025/09/29 21:29:46 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 21:29:46 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

Server kann jetzt über den Port 5003 (http://127.0.0.1:5003) im Browser gesehen werden.
Falls gewünscht zeigt es die nächste Zeile direkt im Notebook an.

In [5]:
# Warten bis Server bereit ist
time.sleep(3)

# Anzeige als iFrame
display(IFrame("http://127.0.0.1:5003", width="100%", height=1000))

# Hinweis: Am Ende des Workshops nicht vergessen zu stoppen mit:
# ui_proc.terminate()


2025/09/29 21:30:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 21:30:33 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/09/29 21:30:35 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 21:30:35 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/09/29 21:30:52 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 21:30:52 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional D

## 2) Experiment anlegen

MlFlow bei Tracking immer alles in Runs.
Ein Experiment ist eine Sammlung von Runs.
Ein Experiment hat eine eindeutige ID selbst wenn andere mit dem gleichen Namen erzeugt werden

In [6]:
# Einheitlicher Experiment-Name für alle Runs
EXPERIMENT_NAME = "sustAInable-mlflow-workshop"
mlflow.set_experiment(EXPERIMENT_NAME)

expereriment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

print("Experiment ID:", expereriment_id)
print("Tracking URI:", mlflow.get_tracking_uri())
print("Experiment:", EXPERIMENT_NAME)


2025/09/29 21:31:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 21:31:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/09/29 21:31:51 INFO mlflow.tracking.fluent: Experiment with name 'sustAInable-mlflow-workshop' does not exist. Creating a new experiment.


Experiment ID: 1
Tracking URI: sqlite:///mlflow_workshop.db
Experiment: sustAInable-mlflow-workshop



## 3) Datensatz: Simpler PV-Datensatz
Daten sind über auch über die URL verfügbar.


#### 3.1) Datensatz als ganzes Eintrage

 Vorteil: Erzeugter Datenhash ist direkt mit dem der Quelle vergleichbar ohne erneutes Speichern.

In [7]:
# Datensatz laden
df = pd.read_csv("pv_data.csv")

# Daten zu run hinzufügen
TARGET = "Erzeugung[Wh] t+24"
SOURCE = "./pv_data.csv"

with mlflow.start_run(run_name="PV Vorhersage - Daten") as run:
    ds_train = mlflow.data.from_pandas(
        df, source=f"{SOURCE}", name="pv-data", targets=TARGET
    )
    mlflow.log_input(ds_train)


#### 3.2) Datensatz Trainings, Validations und Testsplits eintragen

Vorteil: Änderungen in den Split sind schneller offensichtlich.

In [8]:
def log_splits(run_name, nested=False):
    with mlflow.start_run(run_name=run_name, nested=nested) as run:
        for split in ["training", "validation", "test"]:
            df = pd.read_csv(f"pv_data-{split}.csv")
            df = mlflow.data.from_pandas(
                df, source=f"pv_data-{split}.csv", name=f"pv-data-{split}", targets=TARGET
            )
            mlflow.log_input(df, context=split)
        run_id = run.info.run_id
        return run_id

run_name = "PV Vorhersage - Daten Splits"
run_id = log_splits(run_name)


## 4) **Training**

Wir werden über alle Experimente immer mit der gleichen Funktion das Model trainieren

In [10]:
# definition Trainingsfunktion
def train_model(run_name, params):
        
        ds_train = pd.read_csv("pv_data-training.csv")
        # build features/target for training
        X_train = ds_train.drop(columns=TARGET)
        y_train = ds_train[TARGET]

        mlflow.log_params(params)
        # Modell trainieren
        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)
        # Modell loggen
        y_train_pred = model.predict(X_train)
        mlflow.sklearn.log_model(
            sk_model=model,
            name="random-forest-model",
            #registered_model_name="RandomForestPVModel",
            input_example=X_train.iloc[:5],
            signature=mlflow.models.infer_signature(X_train, y_train_pred),
        )


        for split in ["training", "validation", "test"]:
            # Trainingsdaten laden
            ds = pd.read_csv(f"pv_data-{split}.csv")
            X = ds.drop(columns=TARGET)
            y = ds[TARGET]
            # Vorhersagen machen
            y_pred = model.predict(X)

            # Metriken berechnen
            rmse = root_mean_squared_error(y, y_pred)

            mlflow.log_metric(f"rmse_{split}", rmse)


#### 4.1) Einzelnes Modell trainieren
Wir trainieren ein einzelnes Random Forest Modell. Wir loggen:
- **Parameter** (Anzahle der Bäume, Tiefe der Bäume, festen Zufallswert)
- **Metriken** (RMSE auf Train/Val/Test)
- **Modell** inkl. Signatur (für spätere Nutzung/Serving)

In [ ]:
run_name = "PV Vorhersage - Einzel Modell"
# Defintion der Paramter für den Run
params = {"n_estimators": 10, "max_depth": 3, "random_state": 7}

run_id = log_splits(run_name)
with mlflow.start_run(run_id=run_id) as run:
    train_model(run_name, params)

2025/09/29 21:35:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 21:35:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/09/29 21:35:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 21:35:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/09/29 21:36:06 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 21:36:06 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


#### 4.2) C02 Footprint
Zusätzlich loggen wir den C02 Footprint des Trainings. Hierfür nutzen wir die CodeCarbon-Bibliothek.
Es gibt dazu viele Alternativen, z.B:
 - CarbonTracker
 - Experiment-Impact-Tracker (EIT)
 - Green-Algorithms
 - Eco2AI


In [ ]:
run_name = "PV Vorhersage - CO2 Footprint"
# Defintion der Paramter für den Run
params = {"n_estimators": 10, "max_depth": 3, "random_state": 7}

#######################
# CO2 Tracking
tacker = EmissionsTracker()
tacker.start()


run_id = log_splits(run_name)
with mlflow.start_run(run_id=run_id) as run:
    train_model(run_name, params)

    ########################
    # CO2 Tracking
    tacker.stop()
    mlflow.log_metric("co2_footprint_kg", tacker.final_emissions)

[codecarbon INFO @ 21:38:29] [setup] RAM Tracking...
[codecarbon INFO @ 21:38:29] [setup] GPU Tracking...
[codecarbon INFO @ 21:38:29] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:38:29] [setup] CPU Tracking...
[codecarbon WARNING @ 21:38:29] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 21:38:29] CPU Model on constant consumption mode: AMD Ryzen 7 3700X 8-Core Processor
[codecarbon INFO @ 21:38:29] >>> Tracker's metadata:
[codecarbon INFO @ 21:38:29]   Platform system: Linux-6.13.4-200.fc41.x86_64-x86_64-with-glibc2.40
[codecarbon INFO @ 21:38:29]   Python version: 3.10.18
[codecarbon INFO @ 21:38:29]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 21:38:29]   Available RAM : 31.246 GB
[codecarbon INFO @ 21:38:29]   CPU count: 16
[codecarbon INFO @ 21:38:29]   CPU model: AMD Ryzen 7 3700X 8-Core Processor
[codecarbon INFO @ 21:38:29]   GPU count: 1
[codecarbon INFO @ 21:38:29]   GPU model: 1 x NVIDIA GeForce RTX 4080 SUPER
[codecarbon INFO @ 

#### 4.3) Hyperparamteroptimierung und Unterruns

Wir wollten eine ganze Versuchsreihe an Experimenten trainieren und diese strukturiert speichern.

In [13]:
run_name = "PV Vorhersage - CO2 Footprint"
# Defintion der Paramter für den Run
params = {"n_estimators": 100, "max_depth": 10, "random_state": 7}

run_id = log_splits(run_name)
tacker = EmissionsTracker()
tacker.start()
# Eltern run starten
with mlflow.start_run(run_id=run_id) as run:
    # 3 Unterruns mit verschiedenen Parametern
    for run_num, n_estimators, max_depth, random_state in [(1, 20, 3, 7), (2, 50, 5, 7), (3, 200, 15, 7)]:
        params = {
            "n_estimators": n_estimators,
            "max_depth": max_depth,
            "random_state": random_state,
        }
        # Unterrun starten
        local_tacker = EmissionsTracker()
        local_tacker.start()
        run_name=f"Unterrun_{run_num}"
        child_run_id = log_splits(run_name,nested=True)
        with mlflow.start_run(run_id=child_run_id, nested=True) as child_run:
            train_model(run_name, params)
            local_tacker.stop()
            mlflow.log_metric("co2_footprint_kg", local_tacker.final_emissions)

        print(f"Unterrun {run_num} mit {n_estimators} Bäumen, max_depth {max_depth} abgeschlossen.")
    tacker.stop()
    mlflow.log_metric("co2_footprint_kg", tacker.final_emissions)

[codecarbon INFO @ 21:40:28] [setup] RAM Tracking...
[codecarbon INFO @ 21:40:28] [setup] GPU Tracking...
[codecarbon INFO @ 21:40:28] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:40:28] [setup] CPU Tracking...
[codecarbon WARNING @ 21:40:28] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 21:40:28] CPU Model on constant consumption mode: AMD Ryzen 7 3700X 8-Core Processor
[codecarbon INFO @ 21:40:28] >>> Tracker's metadata:
[codecarbon INFO @ 21:40:28]   Platform system: Linux-6.13.4-200.fc41.x86_64-x86_64-with-glibc2.40
[codecarbon INFO @ 21:40:28]   Python version: 3.10.18
[codecarbon INFO @ 21:40:28]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 21:40:28]   Available RAM : 31.246 GB
[codecarbon INFO @ 21:40:28]   CPU count: 16
[codecarbon INFO @ 21:40:28]   CPU model: AMD Ryzen 7 3700X 8-Core Processor
[codecarbon INFO @ 21:40:28]   GPU count: 1
[codecarbon INFO @ 21:40:28]   GPU model: 1 x NVIDIA GeForce RTX 4080 SUPER
[codecarbon INFO @ 

Unterrun 1 mit 20 Bäumen, max_depth 3 abgeschlossen.


[codecarbon INFO @ 21:40:42] Energy consumed for RAM : 0.000007 kWh. RAM Power : 11.717405319213867 W
[codecarbon INFO @ 21:40:42] Energy consumed for all GPUs : 0.000014 kWh. Total GPU Power : 23.101 W
[codecarbon INFO @ 21:40:42] Energy consumed for all CPUs : 0.000019 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 21:40:42] 0.000040 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:40:42] [setup] RAM Tracking...
[codecarbon INFO @ 21:40:42] [setup] GPU Tracking...
[codecarbon INFO @ 21:40:42] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:40:42] [setup] CPU Tracking...
[codecarbon WARNING @ 21:40:42] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 21:40:42] CPU Model on constant consumption mode: AMD Ryzen 7 3700X 8-Core Processor
[codecarbon INFO @ 21:40:42] >>> Tracker's metadata:
[codecarbon INFO @ 21:40:42]   Platform system: Linux-6.13.4-200.fc41.x86_64-x86_64-with-glibc2.40
[codecarbon INFO @ 21:40:42]   Python version

Unterrun 2 mit 50 Bäumen, max_depth 5 abgeschlossen.


[codecarbon INFO @ 21:40:46] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.717405319213867 W
[codecarbon INFO @ 21:40:46] Energy consumed for all GPUs : 0.000098 kWh. Total GPU Power : 23.469 W
[codecarbon INFO @ 21:40:46] Energy consumed for all CPUs : 0.000135 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 21:40:46] 0.000282 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:40:50] Energy consumed for RAM : 0.000016 kWh. RAM Power : 11.717405319213867 W
[codecarbon INFO @ 21:40:50] Energy consumed for all GPUs : 0.000031 kWh. Total GPU Power : 22.360000000000003 W
[codecarbon INFO @ 21:40:50] Energy consumed for all CPUs : 0.000045 kWh. Total CPU Power : 32.5 W
[codecarbon INFO @ 21:40:50] 0.000093 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:40:50] Energy consumed for RAM : 0.000062 kWh. RAM Power : 11.717405319213867 W
[codecarbon INFO @ 21:40:50] Energy consumed for all GPUs : 0.000122 kWh. Total GPU Power : 22.360000000000003 W
[co

Unterrun 3 mit 200 Bäumen, max_depth 15 abgeschlossen.


#### 4.4) Zusätzliche Evaluationsmöglichkeiten hinzufügen

Wir wollten eine ganze Versuchsreihe an Experimenten nachträglich noch mit Feature-Importance Plots anreichern.

In [14]:
client = MlflowClient()
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

with mlflow.start_run(run_id=run_id):
    child_runs = mlflow.search_runs(
        experiment_ids=[exp.experiment_id],
        filter_string=f"tags.mlflow.parentRunId = '{run_id}'",
        order_by=["metrics.rmse_validation ASC"],
    )

    for child_run_id in child_runs.run_id:
        # attach to the existing child run so artifacts land there
        with mlflow.start_run(run_id=child_run_id, nested=True):
            # 1) load the model from this child run
            model_uri = f"runs:/{child_run_id}/random-forest-model"
            model = mlflow.sklearn.load_model(model_uri)

            # 2) get feature names from training split
            df_train = pd.read_csv("pv_data-training.csv")
            X_train = df_train.drop(columns=[TARGET])    # features only

            # 3) prepare artifact dir & write CSV
            art_dir = Path(mlflow.get_artifact_uri().replace("file://", ""))
            art_dir.mkdir(parents=True, exist_ok=True)

            importances = getattr(model, "feature_importances_", None)
            if importances is None:
                raise AttributeError("Model has no feature_importances_. Did you log a tree model?")

            names = list(getattr(X_train, "columns", [f"x{j}" for j in range(len(importances))]))
            out_path = art_dir / "feature_importances.csv"

            pd.DataFrame({"feature": names, "importance": importances}) \
              .sort_values("importance", ascending=False) \
              .to_csv(out_path, index=False)

            mlflow.log_artifact(str(out_path))

        # ---- Plot
        sorted_idx = importances.argsort()[::-1]  # descending
        plt.figure(figsize=(10, 6))
        plt.bar(range(len(importances)), importances[sorted_idx], align="center")
        plt.xticks(range(len(importances)), [names[i] for i in sorted_idx], rotation=90)
        plt.title("Feature Importances")
        plt.tight_layout()

        out_png = art_dir / "feature_importances.png"
        plt.savefig(out_png, dpi=150)
        plt.close()
        mlflow.log_artifact(str(out_png))

NameError: name 'Path' is not defined


## 5) **Deploy‑Vorbereitung** – Bestes Modell in die **Model Registry**
Wir registrieren das beste Modell als `RegisteredModel` und promoten es in **Staging**.  

#### 5.1) Runs vergleichen und bestes Modell finden
In der MLflow‑UI könnt ihr visuell vergleichen. Hier zeigen wir zusätzlich, wie man **programmatisch** den besten Run (min `rmse_val`) auswählt.

In [78]:
client = MlflowClient()
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

child_runs = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    filter_string=f"tags.mlflow.parentRunId = '{run_id}'",
    order_by=["metrics.rmse_validation ASC"],
)

best_row = child_runs.iloc[0]
best_run_id = best_row["run_id"]

# metric column names in search_runs DF are prefixed with "metrics."
metric_col = (
    "metrics.rmse_validation"
    if "metrics.rmse_validation" in child_runs.columns
    else "metrics.rmse_val"
)

best_rmse = best_row[metric_col]
print("Best run:", best_run_id, "RMSE:", best_rmse)


Best run: f27269bd4e3b49c28eb531a26212c888 RMSE: 470.9532302172802


#### 5.2) Das beste Model registrieren
Wir nutzen den Staging Tag das Model noch über die REST API getestet werden muss bevor wir es auf Produktive setzen

In [80]:
import warnings


client = MlflowClient()

REGISTERED_NAME = "sustainable-random-forest"

# Modellartefakt-Pfad des besten Runs
model_uri = f"runs:/{best_run_id}/random-forest-model"

# Registrierung (legt ggf. automatisch die erste Version an)
mv = mlflow.register_model(model_uri=model_uri, name=REGISTERED_NAME)
print("Registered:", mv.name, "version:", mv.version)

# In STAGING promoten
client.transition_model_version_stage(
    name=REGISTERED_NAME,
    version=mv.version,
    stage="Staging",
    archive_existing_versions=False,
)
print("Promoted to:", "Staging")


Registered model 'sustainable-random-forest' already exists. Creating a new version of this model...
2025/09/29 02:13:20 WARNING mlflow.tracking._model_registry.fluent: Run with id f27269bd4e3b49c28eb531a26212c888 has no artifacts at artifact path 'random-forest-model', registering model based on models:/m-e7ab70b948b14e35a38693f4d189622c instead


Registered: sustainable-random-forest version: 1
Promoted to: Staging


Created version '1' of model 'sustainable-random-forest'.



## 6) **Serve** – REST‑API starten & Prediction testen

Wir testen erstmal mit unserem Notebook ob das Model über den REST-Server funktioniert, falls ja, setzen wir es auf Produktion 


##### 6.1.2 Server starten (Python)


In [81]:
import subprocess, time

# Name of the registered model
REGISTERED_NAME = "sustainable-random-forest"

# Start serving the "Staging" version in background
serve_proc = subprocess.Popen(
    [
        "mlflow", "models", "serve",
        "-m", f"models:/{REGISTERED_NAME}/Staging",
        "-p", "5001", "--no-conda"
    ]
)

# give server some time to start
time.sleep(5)
print("Model server is running at http://127.0.0.1:5001")
print("Send POST requests to /invocations")

# When you're done:
# serve_proc.terminate()

2025/09/29 02:13:28 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 02:13:28 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/09/29 02:13:28 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/29 02:13:28 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/09/29 02:13:28 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/09/29 02:13:28 INFO mlflow.pyfunc.backend: === Running command 'exec uvicorn --host 127.0.0.1 --port 5001 --workers 1 mlflow.pyfunc.scoring_server.app:app'
2025/09/29 02:13:

Model server is running at http://127.0.0.1:5001
Send POST requests to /invocations


In [86]:
# Test the REST API with a few samples from the test set
MODEL_URI = "models:/sustainable-random-forest/Staging"  # adjust to your model
model = mlflow.pyfunc.load_model(MODEL_URI)

input_schema = model.metadata.get_input_schema()
# Try to pull expected DataFrame column names (if any)
df_cols = [c.name for c in getattr(input_schema, "inputs", []) if hasattr(c, "name") and c.name]

# Load test data (same as used during training)
ds_test =  pd.read_csv("pv_data-test.csv")
X_test = ds_test.drop(columns=[TARGET])
y_test = ds_test[TARGET]

row = X_test[27:30]  # shape (1, d)
# Ensure dtype float (some schemas enforce double/float)
row = np.asarray(row, dtype=float)

if df_cols:  # Model expects a DataFrame with named columns
    # If your X_test has different column order/count -> align or stop with a clear error
    if row.shape[1] != len(df_cols):
        raise ValueError(f"Feature count mismatch: payload has {row.shape[1]} cols, "
                         f"model expects {len(df_cols)}: {df_cols[:5]}...")
    payload = {"dataframe_split": {"columns": df_cols, "data": row.tolist()}}
else:        # Model expects a tensor (list-of-lists), no column names
    # Infer expected feature count from tensor spec (last dim)
    tspec = input_schema.inputs[0]  # TensorSpec
    expected = tspec.shape[-1] if tspec.shape and tspec.shape[-1] is not None else row.shape[1]
    if row.shape[1] != expected:
        raise ValueError(f"Feature count mismatch: payload has {row.shape[1]}, model expects {expected}.")
    payload = {"inputs": row.tolist()}

res = requests.post(
    "http://127.0.0.1:5001/invocations",
    headers={"Content-Type": "application/json"},
    data=json.dumps(payload),
)
print("Request payload:", json.dumps(payload)[:500])
print("Status:", res.status_code)
print("Response:", res.text[:500])

print("Original y_test:", y_test[27:30].values)

print("Terminal command to test with curl:")
print("curl -X POST http://127.0.0.1:5001/invocations -H 'Content-Type: application/json' -d '", json.dumps(payload), "'")


Request payload: {"dataframe_split": {"columns": ["Stunde des Tages sin", "Stunde des Tages cos", "Tag des Jahres sin", "Tag des Jahres cos", "Erzeugung[Wh] t-1", "Erzeugung[Wh] t-3", "Erzeugung[Wh] t-6"], "data": [[-0.887885218402376, 0.460065037731152, 0.952117665910714, -0.305731827359753, 491.0, 2166.0, 3509.0], [0.979084087682323, 0.203456013052634, 0.88545602565321, 0.464723172043769, 0.0, 0.0, 0.0], [0.631087944326053, -0.77571129070442, -0.79247684195109, -0.609902004400073, 550.0, 165.0, 0.0]]}}
Status: 200
Response: {"predictions": [37.26823660237449, 1.1928851058436225, 1256.064442565918]}
Original y_test: [  21.    0. 1345.]
Terminal command to test with curl:
curl -X POST http://127.0.0.1:5001/invocations -H 'Content-Type: application/json' -d ' {"dataframe_split": {"columns": ["Stunde des Tages sin", "Stunde des Tages cos", "Tag des Jahres sin", "Tag des Jahres cos", "Erzeugung[Wh] t-1", "Erzeugung[Wh] t-3", "Erzeugung[Wh] t-6"], "data": [[-0.887885218402376, 0.4600650377

#### 6.2 Terminalversion
Die Modelle können jetzt auf einem Inference Server über das Terminal zur Verfügung gestellt werden
> **Hinweis:** Serving bitte **im Terminal** starten. Öffnet ein zweites Terminal im selben Ordner.

Dieses mal mit dem Production Tag. Dafür in der UI das Model auf Production setzten
(Dafür die neue UI Ansicht ausschalten)

##### 6.2.1 Server starten (Terminal)

```bash
conda activate sustainable-mlflow
export MLFLOW_TRACKING_URI=sqlite:///mlflow_workshop.db
mlflow models serve -m "models:/{REGISTERED_NAME}/Production" -p 5002 --no-conda
```

Also in unserem Fall ()

```bash
mlflow models serve -m "models:/sustainable-random-forest/Staging" -p 5002 --no-conda --host 0.0.0.0
```

Das "--host 0.0.0.0" erlaubt uns von extern darauf zuzugreigen


*Alternativ* könnt ihr auch eine konkrete Version serven:
```bash
mlflow models serve -m "models:/{REGISTERED_NAME}/1" -p 5002 --no-conda
```

##### 6.2.2 Prediction aufrufen (Terminal)
```bash
conda activate sustainable-mlflow
curl -X POST http://127.0.0.1:5002/invocations \
  -H 'Content-Type: application/json' \
  -d '{"dataframe_split":{"columns":[%COLS%],"data":[%ROW%]}}'
```

```bash
conda activate sustainable-mlflow
curl -X POST http://127.0.0.1:5002/invocations -H 'Content-Type: application/json' -d ' {"dataframe_split": {"columns": ["Stunde des Tages sin", "Stunde des Tages cos", "Tag des Jahres sin", "Tag des Jahres cos", "Erzeugung[Wh] t-1", "Erzeugung[Wh] t-3", "Erzeugung[Wh] t-6"], "data": [[-0.887885218402376, 0.460065037731152, 0.952117665910714, -0.305731827359753, 491.0, 2166.0, 3509.0], [0.979084087682323, 0.203456013052634, 0.88545602565321, 0.464723172043769, 0.0, 0.0, 0.0], [0.631087944326053, -0.77571129070442, -0.79247684195109, -0.609902004400073, 550.0, 165.0, 0.0]]}} '
```